# Implement Automatic Tool Calling with Granite-3 Models in LM Studio

This notebook serves as a practical guide to implementing automatic tool calling with Large Language Models (LLMs) like the Granite-3 series in LM Studio. While LLMs excel at understanding and generating human-like text, they often face limitations when tasks require precise computation, access to real-time external data, or the execution of specific, well-defined procedures. By equipping LLMs with a set of "tools"—essentially external functions they can choose to call—we can significantly extend their capabilities. This tutorial will demonstrate how to define these tools and integrate them, enabling the LLM to perform a wider range of tasks with greater reliability.

### Getting Started with Granite models on LM Studio

[Download LM Studio](https://lmstudio.ai/download) and follow [these instructions](https://lmstudio.ai/docs/app/basics/download-model) to download models to your local machine. We will be using the [granite-3.3-8b-instruct model](https://huggingface.co/ibm-granite/granite-3.3-8b-instruct-GGUF) for this recipe, but feel free to use any LLM of your choice.

We first need to install the necessary libraries:

In [ ]:
%pip install git+https://github.com/ibm-granite-community/utils \
    lmstudio \
    chess

In [ ]:
import lmstudio as lms

And now you can load your desired model and start chatting!

In [ ]:
model = lms.llm("ibm-granite/granite-3.3-8b-instruct-GGUF")

print(model.respond("Hello Granite!"))

### Performing Calculations Without Tools

Let's start by asking the model to do a straightforward calculation.

In [ ]:
print(model.respond("What is 26.97 divided by 6.28? Don't round."))

While the model may be able to provide a close approximation, it won't return the exact answer, because it can't actually calculate the quotient on it's own. 

### Creating Tools

To solve this, we will provide the model some tools. Tools are python functions that we provide to the model at inference. The model can choose to call one or more of these tools to answer the user's query. 

Take a look at the [LM Studio Docs](https://lmstudio.ai/docs/python) for more information on how to write tools. In general, you should make sure your tooling functions have an appropriate name, defined input and output types, and a description that explains the purpose of the tool. All of this information is passed to the model, and can help it select the correct tool to answer your query.

We will write several simple math functions for the model to use as tools: 

In [ ]:
def add(a: float, b:float):
    """Given two numbers a and b, return a + b."""
    return a + b

def subtract(a: float, b:float):
    """Given two numbers a and b, return a - b."""
    return a - b

def multiply(a: float, b: float):
    """Given two numbers a and b, return a * b."""
    return a * b

def divide(a: float, b: float):
    """Given two numbers a and b, return a / b."""
    return a / b

def exp(a: float, b:float):
    """Given two numbers a and b, return a^b"""
    return a ** b


Now, we can rerun the same query, but provide the model some tools to help it answer.

In [ ]:
model.act(
  "What is 26.97 divided by 6.28? Don't round.",
  [add, subtract, multiply, divide, exp],
  on_message=print,
)

The model was able to select the correct tool, and call it to get an exact answer to the question. It was also able to avoid using the irrelevant tools.

### How Many R's in Strawberry?

A very simple question that stumps even the smartest language models. Almost every single LLM with a training cutoff prior to 2024 answers that there are only 2 r's in the word "strawberry". As a bonus, it might even hallucinate incorrect positions for the letters. 

Nowadays LLMs tend to get this specific question right, purely because it's virality landed it in most training datasets. However LLMs still commonly fail on similar letter counting tasks.

In [ ]:
print(model.respond("How many b's are in the word 'blackberry'?"))


Let's write a tool to help the model do a better job.

In [ ]:
def get_letter_frequency(word: str) -> dict:
    """Takes in a word (string) and returns a dictionary containing the counts of each letter that appears in the word. """

    letter_frequencies = {}

    for letter in word:
        if letter in letter_frequencies:
            letter_frequencies[letter] += 1
        else:
            letter_frequencies[letter] = 1

    return letter_frequencies

Now we can pass the tool to the model, and rerun the prompt.

In [ ]:
model.act(
  "How many b's are in the word 'blackberry'?",
  [get_letter_frequency],
  on_message=print,
)

### An Interactive Example ♟️

One of the best use-cases of this automatic tool calling workflow is to give your model the ability to interact with it's external environment.

Let's build an agent that uses tools to play chess!



While language models may have strong conceptual knowledge of chess, they aren't inherently designed to understand a chess board. If you try to play a game of chess with an online chatbot, it will often derail after several turns, making illegal or irrational moves.

We are providing the model several tools that help it understand and interact with the board.

- **get_move_history()**: provides a list of all moves played so far
- **legal_moves()**: provides a list of all legal moves in the current position
- **possible_captures()**: provides a list of all possible captures in the current position
- **possible_checks()**: provides a list of all possible checks in the current position
- **make_ai_move()**: an interface to let the model input it's move

It's not a lot: but it is enough for the model to play a full game of chess without hallucinating, and use some intelligent reasoning to base it's decisions.


In [ ]:
import chess
from IPython.display import display, SVG, clear_output
import random

board = chess.Board()
ai_pos = 0

def legal_moves() -> list[str]:
    """
    Returns a list of legal moves in standard algebraic notation.
    """
    return [board.san(move) for move in board.legal_moves]

def possible_captures() -> list[str]:
    """
    Returns a list of possible captures in standard algebraic notation.
    """
    return [board.san(move) for move in board.generate_legal_captures()]

def possible_checks() -> list[str]:
    """
    Returns a list of possible checks in standard algebraic notation.
    """
    return [board.san(move) for move in board.legal_moves if board.gives_check(move)]

def get_move_history() -> list[str]:
    """
    Returns a list of moves made in the game so far in standard algebraic notation.
    """
    return [board.san(move) for move in board.move_stack]

def is_ai_turn() -> bool:
    return bool(board.turn) == (ai_pos == 0)

def make_ai_move(move: str) -> None:
    """
    Given a string representing a valid move in chess notation, pushes move onto chess board.
    If non-valid move, raises a ValueError with message "Illegal move.
    If called when it is not the AI's turn, raises a ValueError with message "Not AI's turn."
    THIS FUNCTION DIRECTLY ENABLES THE AI TO MAKE A MOVE ON THE CHESS BOARD.
    """
    if is_ai_turn():
        try:
            board.push_san(move)
        except ValueError as e:
            raise ValueError(e)
    else:
        raise ValueError("Not AI's turn.")

def make_user_move(move: str) -> None:
    """
    Given a string representing a valid move in chess notation, pushes move onto chess board.
    If non-valid move, raises a ValueError with message "Illegal move.
    If called when it is not the player's turn, raises a ValueError with message "Not player's turn."
    If valid-move, updates the board and displays the current state of the board.
    """
    if not is_ai_turn():
        try:
            board.push_san(move)
        except ValueError as e:
            raise ValueError(e)
    else:
        raise ValueError("Not player's turn.")

def print_fragment(fragment, round_index=0):
    print(fragment.content, end="", flush=True)


Now we can play a match with the AI!

In [ ]:
move = 0
import chess.svg

board.reset()
ai_pos = round(random.random())

def update_board(move = move, ai_pos = ai_pos):
    """
    Updates the chess board display in the notebook.
    """
    clear_output(wait=True)  # Clear previous output
    print(f"Board after move {move+1}")
    if (ai_pos == 1):
        display(SVG(chess.svg.board(board, size=400)))
    else:
        display(SVG(chess.svg.board(board, size=400, orientation = chess.BLACK)))

def get_end_state():
    """
    Returns the end state of the chess game.
    """
    if board.is_checkmate():
        return "Checkmate!"
    elif board.is_stalemate():
        return "Stalemate!"
    elif board.is_insufficient_material():
        return "Draw by insufficient material!"
    elif board.is_seventyfive_moves():
        return "Draw by 75-move rule!"
    elif board.is_fivefold_repetition():
        return "Draw by fivefold repetition!"
    else:
        return None

clear_output(wait=True) # Clear any previous output from the cell
if (ai_pos == 1):
    display(SVG(chess.svg.board(board, size=400)))
else:
    display(SVG(chess.svg.board(board, size=400, orientation = chess.BLACK)))

# 2. Loop through moves, apply each move, clear previous output, and display new board
userEndGame = False
while True:

    if ai_pos == 0:
        # AI's turn
        model.act(
            """
            You are a chess AI, playing for black. Your task is to make the best move in the current position, using the provided tools. You should use your overall chess knowledge, including openings, tactics, and strategies, as your primary method to determine good moves. Use the provided tools as an assistant to improve your understanding of the board state and to make your moves.
            """,
            [get_move_history, legal_moves, possible_captures, possible_checks, make_ai_move],
            max_prediction_rounds = 5,
        )

        if is_ai_turn(): # failsafe in case AI does not make a move
           make_ai_move(legal_moves()[0])  # Default to the first legal move if AI does not respond

        update_board(move)
        move += 1
        game_over_message = get_end_state()
        if game_over_message:
            print(game_over_message)
            break

        # User's turn
        while True:
            user_move = input("User (Playing Black): Input your move. Input 'help' to see the list of possible moves. Input 'quit' to end the game ->")
            if user_move.lower() == 'quit':
                print("Game ended by user.")
                userEndGame = True
                break
            if user_move.lower() == 'help':
                print("Possible moves:", legal_moves())
                continue
            try:
                make_user_move(user_move)
                break
            except ValueError as e:
                print(e)

        if userEndGame:
            break

        update_board(move)
        move += 1
        game_over_message = get_end_state()
        if game_over_message:
            print(game_over_message)
            break
    else:
        # User's turn
        while True:
            user_move = input("User (Playing White): Input your move. Input 'help' to see the list of possible moves. Input 'quit' to end the game ->")
            if user_move.lower() == 'quit':
                print("Game ended by user.")
                userEndGame = True
                break
            if user_move.lower() == 'help':
                print("Possible moves:", legal_moves())
                continue
            try:
                make_user_move(user_move)
                break
            except ValueError as e:
                print(e)

        if userEndGame:
            break

        update_board(move)
        move += 1
        game_over_message = get_end_state()
        if game_over_message:
            print(game_over_message)
            break

        model.act(
            """
            You are a chess AI, playing for black. Your task is to make the best move in the current position, using the provided tools. You should use your overall chess knowledge, including openings, tactics, and strategies, as your primary method to determine good moves. Use the provided tools as an assistant to improve your understanding of the board state and to make your moves.
            """,
            [get_move_history, legal_moves, possible_captures, possible_checks, make_ai_move],
            max_prediction_rounds = 5,
        )

        if is_ai_turn(): # failsafe in case AI does not make a move
           make_ai_move(legal_moves()[0])  # Default to the first legal move if AI does not respond

        update_board(move)
        move += 1
        game_over_message = get_end_state()
        if game_over_message:
            print(game_over_message)
            break

### Summary

In this notebook, we demonstrated how integrating tools can enhance the utility and agentic capability of LLMs. We illustrated that by providing an LLM with access to predefined external functions, it can transcend its core language processing capabilities to perform tasks like accurate calculations or interface with external systems, which it cannot do reliably on its own. The key takeaway is that tool-use empowers LLMs to delegate specific sub-problems to specialized routines, allowing them to ground their responses in factual data or precise operations. This approach not only improves accuracy but also enables LLMs to engage in more complex, interactive workflows, effectively transforming them into more versatile and powerful assistants.